<a href="https://colab.research.google.com/github/kumar11jr/Machine_learning/blob/main/titanic_without_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [128]:
import numpy as np
import pandas as pd


In [129]:
df = pd.read_csv("/content/tested.csv")

In [130]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [131]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin',],inplace=True)

In [132]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,34.5,0,0,7.8292,Q
1,1,3,female,47.0,1,0,7.0000,S
2,0,2,male,62.0,0,0,9.6875,Q
3,0,3,male,27.0,0,0,8.6625,S
4,1,3,female,22.0,1,1,12.2875,S


In [133]:
df = df.dropna(subset=['Fare'])

In [134]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state=42)

In [135]:
x_train.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
287,1,male,24.0,1,0,82.2667,S
387,2,male,57.0,0,0,13.0000,S


In [136]:
df.isnull().sum()

Survived     0
Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         0
Embarked     0
dtype: int64

In [137]:
from sklearn.impute import SimpleImputer


In [138]:
si_age = SimpleImputer(strategy='mean')
si_embarked = SimpleImputer(strategy='most_frequent')

x_train_age = si_age.fit_transform(x_train[['Age']])
x_train_embarked = si_embarked.fit_transform(x_train[['Embarked']])

x_test_age = si_age.fit_transform(x_test[['Age']])
x_test_embarked = si_embarked.fit_transform(x_test[['Embarked']])

In [139]:
from sklearn.preprocessing import OneHotEncoder

In [140]:
ohe_sex = OneHotEncoder(sparse=False,handle_unknown='ignore')
ohe_embarked = OneHotEncoder(sparse=False,handle_unknown='ignore')

x_train_sex = ohe_sex.fit_transform(x_train[['Sex']])
x_train_embarked = ohe_embarked.fit_transform(x_train_embarked)

x_test_sex = ohe_sex.fit_transform(x_test[['Sex']])
x_test_embarked = ohe_embarked.transform(x_test_embarked)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [141]:
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
287,1,male,24.0,1,0,82.2667,S
387,2,male,57.0,0,0,13.0000,S
210,3,male,32.0,0,0,22.5250,S
93,3,male,NaN,0,0,8.0500,S
285,3,male,36.0,0,0,7.2500,S


In [142]:
df.dropna(axis=0,inplace=True)

In [143]:
x_train_rem = x_train.drop(columns=['Sex','Age','Embarked'])


In [144]:
x_test_rem = x_test.drop(columns=['Sex','Age','Embarked'])

In [145]:
x_train_transformed = np.concatenate((x_train_rem,x_train_age,x_train_sex,x_train_embarked),axis=1)
x_test_transformed = np.concatenate((x_test_rem,x_test_age,x_test_sex,x_test_embarked),axis=1)

In [146]:
np.any(np.isnan(x_train_transformed))

False

In [147]:
from sklearn.tree import DecisionTreeClassifier


In [148]:
model = DecisionTreeClassifier()

In [149]:
model.fit(x_train_transformed,y_train)

DecisionTreeClassifier()

In [150]:
y_pred = model.predict(x_test_transformed)

In [151]:
model.score(x_test_transformed,y_test)

1.0

In [152]:
import pickle


In [157]:
pickle.dump(ohe_sex,open('models/ohe_sex.pkl','wb'))
pickle.dump(ohe_embarked,open('models/ohe_embarked.pkl','wb'))
pickle.dump(model,open('models/model.pkl','wb'))